In [1]:
## download all by 1
import requests
from bs4 import BeautifulSoup
import os
import json
import pprint as pp
import time
from random import randint
from time import sleep
import urllib.request 

path_resource = r'./logo-resource_byyear/'
if not os.path.exists(path_resource):
    os.mkdir(path_resource)
    
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'} 

#異議審定書
url_start = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=5f1da434-ad44-4955-97d3-1ac4433603f1(203.204.81.115)'
#評定書
url_start2 = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=e6c2b148-1bdd-459d-ba7d-45bc5b1f41e4(111.250.205.58)'
#廢止處分書
url_start3 = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=09fdc206-3f0d-4407-ae3f-a796abfd3537(111.250.205.58)'
#核駁審定書
url_start4 = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=6fecaded-2523-4948-8c92-6583be0b4849(111.250.205.58)'
#適用法條查詢
url_start5 = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=60e97d62-be61-4c37-b8da-dbcd050c4d10(111.250.205.58)'

url_each = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp'

# year period select
for years in range(107,108):
    try:
        data_fore = {'state': 'queryResultJSONObject',
        'searchConditions': '{"datePubStart":"%s/01/01","datePubEnd":"%s/06/31","dptKind":"OPP","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'%(years,years)
               }
        data_back = {'state': 'queryResultJSONObject',
        'searchConditions': '{"datePubStart":"%s/07/01","datePubEnd":"%s/12/31","dptKind":"OPP","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'%(years,years)
               }
        res_fore = requests.post(url_start, headers=headers, data=data_fore)
        res_back = requests.post(url_start, headers=headers, data=data_back)
        list_fore = json.loads(res_fore.text)
        list_back = json.loads(res_back.text)
        item_back = 0
        item_fore = 0
        d1 = []
        d2 = []
        
        # crawler fore year data
        for i in list_fore['receiveData']['resultArr']:
            try:
                kind_fore = i['DPT_KIND']
                mark_fore = i['MARK_NAME']
                no_fore = i['PUB_NO']
                title_fore = no_fore[-12:-1]
#                 print(title_fore)
                file_fore = i['FILENAME']
                data_sfore = {'state': 'getContentByFileName',
                            'filename': file_fore
                             }

                # crawler content
                res_sec_fore = requests.post(url_each, headers=headers, data=data_sfore)
                raw_content_fore = json.loads(res_sec_fore.text)
                content_fore = raw_content_fore['content']['content']

                #crawler image
                case_no = raw_content_fore['content']['main'][77:118]
                url_img = 'https://twtmsearch.tipo.gov.tw/SS0/SS0201_q.jsp?q=1&state=doQuery&showType=2&caseType=1&caseNo=%s&caseNo2=&isReadBulletinen_US=&isReadBulletinzh_TW=true&l6=zh_TW' %(case_no)
                res_img = requests.get(url_img, headers=headers)
                raw_img = json.loads(res_img.text)['datas']['SCN2']['fullFileName']

                url_images = 'https://twtmsearch.tipo.gov.tw/imageLoad.jsp?path=%s&formatName=jpeg&pathCodeId=282_pic'%(raw_img)

                # save image.
                if not os.path.exists('%s/%s/%s.jpg' %(path_resource,years,title_fore)):
                    urllib.request.urlretrieve(url_images,'%s/%s/%s.jpg' %(path_resource,years,title_fore))

                # set result to dict and convert to json.
                
                d1.append({"title":title_fore,"markname":mark_fore,"kind":kind_fore,"url_images":url_images,"content": content_fore})
    
                item_fore +=1
                if years==102:
                    print(years,'fore: ',item_fore,"/399", title_fore)
                if years==103:
                    print(years,'fore: ',item_fore,"/349", title_fore)
                if years==104:
                    print(years,'fore: ',item_fore,"/384", title_fore)
                if years==105:
                    print(years,'fore: ',item_fore,"/359", title_fore)
                if years==106:
                    print(years,'fore: ',item_fore,"/307", title_fore)
                if years==107:
                    print(years,'fore: ',item_fore,"/386", title_fore)
                if years==108:
                    print(years,'fore: ',item_fore,"/384", title_fore)
                if years==109:
                    print(years,'fore: ',item_fore,"/273", title_fore)

            except TypeError as e:
                print('==========')
                print(title_fore,': no content found')
                print(e.args)
                print('==========')
                continue
            except json.JSONDecodeError as e:
                print('==========')
                print(title_fore, ':no page found')
                print(e.args)
                print('==========')
                continue
        try:
            with open('%s/%s_fore.json' %(path_resource,years), 'w', encoding='utf-8') as f:
                json.dump(d1, f, ensure_ascii=False)
        except FileNotFoundError as e:
            no_fore = no_fore.replace('/', '-')
            with open('%s/%s_fore.json' %(path_resource,years), 'w', encoding='utf-8') as f:
                json.dump(d1, f, ensure_ascii=False)
        except OSError:
            pass
        print("save json complete!===========================================================")
        
        #sleep range (from 1 to 7)
        sleep(randint(60,65))

        # crawler back year data
#         for j in list_back['receiveData']['resultArr']:
#             try:
#                 kind_back = j['DPT_KIND']
#                 mark_back = j['MARK_NAME']
#                 file_back = j['FILENAME']
#                 no_back = j['PUB_NO']
#                 title_back = no_back[-12:-1]
#                 data_sback = {'state': 'getContentByFileName',
#                             'filename': file_back
#                              }
#                 res_sec_back = requests.post(url_each, headers=headers, data=data_sback)

#                 raw_content_back = json.loads(res_sec_back.text)
#                 content_back = raw_content_back['content']['content']


#                 #crawler image
#                 case_no2 = raw_content_back['content']['main'][77:118]
#                 url_img2 = 'https://twtmsearch.tipo.gov.tw/SS0/SS0201_q.jsp?q=1&state=doQuery&showType=2&caseType=1&caseNo=%s&caseNo2=&isReadBulletinen_US=&isReadBulletinzh_TW=true&l6=zh_TW' %(case_no2)
#                 res_img2 = requests.get(url_img2, headers=headers)
#                 raw_img2 = json.loads(res_img2.text)['datas']['SCN2']['fullFileName']

#                 url_images2 = 'https://twtmsearch.tipo.gov.tw/imageLoad.jsp?path=%s&formatName=jpeg&pathCodeId=282_pic'%(raw_img2)

#                 if not os.path.exists('%s/%s/%s.jpg' %(path_resource,years,title_back)):
#                     urllib.request.urlretrieve(url_images2,'%s/%s/%s.jpg' %(path_resource,years,title_back))

#                 # set result to dict format
#                 d2.append({"title":title_back,"mark_name":mark_back,"kind":kind_back,"url_images": url_images2,"content": content_back})
                
#                 item_back +=1
                
#                 if years==102:
#                     print(years,'back: ',item_back,"/496", title_back)
#                 if years==103:
#                     print(years,'back: ',item_back,"/410", title_back)
#                 if years==104:
#                     print(years,'back: ',item_back,"/374", title_back)
#                 if years==105:
#                     print(years,'back: ',item_back,"/381", title_back)
#                 if years==106:
#                     print(years,'back: ',item_back,"/342", title_back)
#                 if years==107:
#                     print(years,'back: ',item_back,"/429", title_back)
#                 if years==108:
#                     print(years,'back: ',item_back,"/358", title_back)
#                 if years==109:
#                     print(years,'back: ',item_back,"/192", title_back)


#             except TypeError as e:
#                 print('==========')
#                 print(title_back,': no content found')
#                 print(e.args)
#                 print('==========')
#                 continue
#             except json.JSONDecodeError as e:
#                 print('==========')
#                 print(title_back, ':no page found')
#                 print(e.args)
#                 print('==========')
#                 continue
#         try:
#             with open('%s/%s_back.json' %(path_resource,years), 'w', encoding='utf-8') as f:
#                 json.dump(d2, f, ensure_ascii=False)
#         except FileNotFoundError as e:
#             no_back = no_back.replace('/', '-')
#             with open('%s/%s_back.json' %(path_resource,years), 'w', encoding='utf-8') as f:
#                 json.dump(d2, f, ensure_ascii=False)
#         except OSError:
#             pass
#         print("save json complete!===========================================================")
#         #sleep range (from 1 to 7)
#         sleep(randint(1,4))

    except json.JSONDecodeError as e:
        print('==========')
#         print(res_fore.text)
        print(e.args)
        print('==========')
        continue
# sleep range (from 1 to 7)
# print(item)
# sleep(randint(1,7))

107 fore:  1 /386 10780343730
107 fore:  2 /386 10780343740
107 fore:  3 /386 10780343800
107 fore:  4 /386 10780343880
107 fore:  5 /386 10780344420
107 fore:  6 /386 10780344430
107 fore:  7 /386 10780344490
107 fore:  8 /386 10780344700
107 fore:  9 /386 10780344720
107 fore:  10 /386 10780344750
107 fore:  11 /386 10780344760
107 fore:  12 /386 10780344800
107 fore:  13 /386 10780345020
107 fore:  14 /386 10780346400
107 fore:  15 /386 10780346430
107 fore:  16 /386 10780346480
107 fore:  17 /386 10780342230
107 fore:  18 /386 10780343120
107 fore:  19 /386 10780343270
107 fore:  20 /386 10780343450
107 fore:  21 /386 10780341170
107 fore:  22 /386 10780341210
107 fore:  23 /386 10780341320
107 fore:  24 /386 10780342130
107 fore:  25 /386 10780342160
107 fore:  26 /386 10780339860
107 fore:  27 /386 10780339980
107 fore:  28 /386 10780339990
107 fore:  29 /386 10780340030
107 fore:  30 /386 10780340630
107 fore:  31 /386 10780340660
107 fore:  32 /386 10780340720
107 fore:  33 /38